<a href="https://colab.research.google.com/github/leolani/cltl-face-all/blob/master/examples/colab/5.friends2gmrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone the repo, download the necessary files, etc.

In [ ]:
%cd /content
!git clone https://github.com/leolani/cltl-face-all
!cd cltl-face-all/ && pip install .
!pip install omegaconf==2.0.5
!pip install tqdm av

# Download the annotations and stuff
!wget https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datatsets-jsons/dataset-small.json
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datatsets-jsons/dataset-medium.json
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datatsets-jsons/dataset-large.json
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datatsets-jsons/datasets.json

!gdown --id 1-2LeHC_5Cm2gWWT6vBrVhp8jorbjkN1_
!unzip visual-features.zip
!rm visual-features.zip

!gdown --id 16ck7plW9v9eSHGCs5wuB2AhhufPRt3Wi
!unzip smaller-dataset.zip
!rm smaller-dataset.zip

!ls smaller-dataset/  |  wc -l
!ls visual-features | wc -l

!git clone https://github.com/cltl/GRMCAnnotation.git
!cd GRMCAnnotation && pip install .

/content
fatal: destination path 'cltl-face-all' already exists and is not an empty directory.
Processing /content/cltl-face-all
  Created wheel for cltl-face-all: filename=cltl_face_all-0.0.0-cp36-none-any.whl size=53234 sha256=13278db5e1686606fcde53809f868d569fdeb6cd1db15843ac1c17f70e6db46a
  Stored in directory: /root/.cache/pip/wheels/8d/71/f3/2db0b17856a0b395aaa04c8562716e0350cfc712cdd4b9947e
Successfully built cltl-face-all
  Found existing installation: cltl-face-all 0.0.0
    Uninstalling cltl-face-all-0.0.0:
      Successfully uninstalled cltl-face-all-0.0.0
--2020-12-01 17:16:32--  https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1119290 (1.1M) [text/plain]
Saving to: 

## read pre-computed visual features and annotations

In [ ]:
import json

with open('datasets.json', 'r') as stream:
    datasets = json.load(stream)

datasets = datasets['large']

from glob import glob
import numpy as np
import os
visual_features = glob('visual-features/*.npy')
visual_features = {os.path.basename(vf).split('.npy')[0] : np.load(vf, allow_pickle=True).item() for vf in visual_features}

## Select a random dialogue. Keep running the cell until you get what you want. Preferably, you want to have a video that includes Joey, Chandler, Monica, Ross, Rachel, or Phoebe.

In [ ]:
from glob import glob
import random
import os
import json
import numpy as np
import av
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import HTML
from base64 import b64encode

VISUAL_FEATURES_PATH = 'visual-features/'
VIDEOS_PATH = "smaller-dataset/"

In [ ]:
def sort_nicely(arg):
    max_length = max([len(arg_) for arg_ in arg])
    min_length = min([len(arg_) for arg_ in arg])

    if max_length == min_length:
        return sorted(arg)
    else:
        return sorted([arg_ for arg_ in arg if len(arg_) == min_length]) + sorted([arg_ for arg_ in arg if len(arg_) == max_length])

diautt = random.choice([foo for foo, bar in datasets['train'].items()])
dia = diautt.split('_')[0]
diautts = sorted([diautt for diautt in list(datasets['train'].keys()) if dia in diautt])
diautts = sort_nicely(diautts)

annots = {diautt: datasets['train'][diautt] for diautt in diautts}
vids_path = {diautt: os.path.join(VIDEOS_PATH, diautt) +  '.mp4' for diautt in diautts}
features_path = {diautt: os.path.join(VISUAL_FEATURES_PATH, diautt) +  '.npy' for diautt in diautts}
features = {diautt: np.load(path, allow_pickle=True).item() for diautt, path in features_path.items()}

an = annots[diautts[0]]
vp = vids_path[diautts[0]]
print(an)
mp4 = open(vp,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=1280 controls>
    <source src="%s" type="video/mp4">
</video>
""" % data_url)

{'SrNo': '10029', 'Utterance': 'Would you look at this dump? He hated us. This is his final revenge!', 'Speaker': 'Monica', 'Emotion': 'disgust', 'Sentiment': 'negative', 'Dialogue_ID': '1000', 'Utterance_ID': '0', 'Season': '2', 'Episode': '3', 'StartTime': '00:06:45,780', 'EndTime': '00:06:49,783'}


In [ ]:
import pickle
from datetime import datetime, timedelta

with open('cltl-face-all/examples/friends-time/friends-time.pkl', 'rb') as stream:
    friends_time = pickle.load(stream)

In [ ]:
import os
import av
import numpy as np
import cv2
from tqdm.notebook import tqdm
import time
import csv

os.makedirs(os.path.join(dia, 'image'), exist_ok=True)
os.makedirs(os.path.join(dia, 'text'), exist_ok=True)
os.makedirs(os.path.join(dia, 'audio'), exist_ok=True)

chat = []
for diautt in tqdm(diautts):
    annot = annots[diautt]
    vis = visual_features[diautt]
    vidpath = vids_path[diautt]

    season = annot['Season']
    episode = annot['Episode']
    emotion = annot['Emotion']
    sentiment = annot['Sentiment']
    starttime = annot['StartTime']
    endttime = annot['EndTime']
    utterance = annot['Utterance']
    speaker = annot['Speaker']

    hours, minutes, seconds = starttime.split(':')
    seconds, milliseconds = seconds.split(',')

    hours, minutes, seconds, milliseconds = int(hours), int(minutes), int(seconds), int(milliseconds)

    time_datetime = friends_time[season][episode] + timedelta(hours=hours, minutes=minutes, seconds=seconds)
    time_unix = time.mktime(time_datetime.timetuple())
    time_unix_ms = int(time_unix*1000 + milliseconds)

    chat.append([speaker, utterance, time_unix_ms])

    aud = diautt + '.wav'
    !ffmpeg -i $vidpath -q:a 0 -map a $dia/audio/$aud

    # container = av.open(vidpath)
    # fps = float(container.streams.video[0].average_rate)
    # for frame in container.decode(video=0):
    #     idx = frame.index
    #     numpy_RGB = np.array(frame.to_image())
    #     numpy_BGR = cv2.cvtColor(numpy_RGB, cv2.COLOR_RGB2BGR)
    #     impath = os.path.join(dia, 'image', diautt + f'_frame{str(idx).zfill(5)}.jpg')
    #     cv2.imwrite(impath, numpy_BGR)
    #     frames[idx] = numpy_RGB

with open(os.path.join(dia, 'text', 'chat.csv'), 'w') as stream:
    stream.write('speaker,utterance,time\n')

    for line in chat:
        print(len(line))
        speaker, utterance, time_unix_ms = line
        stream.write(speaker)
        stream.write(',')
        stream.write(f"\"{utterance}\"")
        stream.write(',')
        stream.write(str(time_unix_ms))
        stream.write('\n')

    

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:

diautt + '.mp3'

'dia1000_utt3.mp3'

In [ ]:
vidpath

'smaller-dataset/dia1000_utt3.mp4'